This notebook shows how to retreieve alerts belonging to either a topic (permanent) or query (temporary) archive token. The alerts are then propagated to a sample T0 filter unit.

In [ ]:
import requests, os

In [ ]:
token = os.environ["ARCHIVE_TOKEN"]
header = {"Authorization": "bearer "+token}
base_url = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3'
from ampel.log.AmpelLogger import AmpelLogger
from ampel.contrib.hu.t0.SimpleDecentFilter import SimpleDecentFilter
from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader
from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier

### 1. Using a query token
In this case a _resume token_ has been produced (and not consumed) e.g. as in `ampel_api_alertquery.ipynb`. 

In [ ]:
# Copy from query
resume_token = ''
# Skip to Sec 3.

### 2. Using a topic token
These are permanent pointers to list of alerts. Here we will first create a stream from this list. 

In [ ]:
# This is a token e.g. from api_create_topic.ipynb
topic_token = 'IbuZVkaJsYXFtuzuO15LHwGBpB4O3RLJTNvKZI2D-88'

In [ ]:
body = {
    "topic": topic_token, 
    "chunk_size": 100
}

In [ ]:
response = requests.post(f"{base_url}/streams/from_topic", headers={"Authorization": "bearer "+token}, json=body )

In [ ]:
if not response.ok:
    print('Accessing stream from topic not successful')
    print(response)

In [ ]:
resume_token = response.json()['resume_token']

### 3. Configure and load a filter

In [ ]:
filter_config = {
    'min_ndet': 1,
    'min_tspan': 0,
    'max_tspan': 50,
    'min_archive_tspan': 0,
    'max_archive_tspan': 500,    
    'min_rb': 0.3,
    'min_drb': 0.995,
    'max_fwhm': 5.5,
    'min_gal_lat': 14,
    'ps1_sgveto_rad': 1,
    'ps1_sgveto_th': 0.8,
    'ps1_confusion_rad': 3,
    'ps1_confusion_sg_tol': 0.1
}

In [ ]:
t0filter = SimpleDecentFilter( **filter_config, logger=AmpelLogger.get_logger() )
t0filter.post_init()

### 4. Iterate through filters from the stream, checking whether the filter would accept them

In [ ]:
config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", 
          "stream": resume_token}   # From above

In [ ]:
alertloader = ZTFArchiveAlertLoader(**config)

In [ ]:
for alert in alertloader.get_alerts():
    print(alert)
    filter_accept = t0filter.process( ZiAlertSupplier.shape_alert_dict( alert, [] ) )
    print(alert['candid'], alert['objectId'], alert['candidate']['jd'], filter_accept)